# Introduction 

In this notebook, we will implement [*Latent Credible Analysis*](https://research.fb.com/publications/latent-credibility-analysis/) models. These are latent probablistic models that use hidden (latent) variables to represents the unknown data source reliabilities and underlying truth values. 

We implement only simpleLCA for now as extension to other models are relatively straight forward.

# SimpleLCA

Here is the plate model of simpleLCA. 

![simpleLCA](./gfx/simpleLCA.png)

### Data 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os.path as op
import numpy as np
import seaborn as sns
import pyro

In [3]:
import sys
sys.path.insert(0, '../')

In [4]:
from spectrum.preprocessing import encoders
from spectrum.judge import lca
from spectrum.judge import utils

In [5]:
DATA_DIR = '../data'
DATA_SET = 'population'

In [6]:
truths = pd.read_csv(op.join(DATA_DIR, DATA_SET, 'truths.csv'))
claims = pd.read_csv(op.join(DATA_DIR, DATA_SET, 'claims.csv'))

In [7]:
truths.shape, claims.shape

((293, 3), (1035, 5))

We decide to model city population as discrete value. Moreover we consider the hidden truth value is only from the set of available assertions. Thus we need to label encode `value` of claims data frame.

### Data Preprocessing 

We need to label encode values of objects in order to feed them to our simpleLCA model

In [8]:
claims_enc, le_dict = encoders.transform(claims)

build the confidence matrix, $[w_{s,o}]$ in the paper, if $w_{s,o} = 1$, then the source s does make an assertion about object o.

In [9]:
mask = lca.build_mask(claims_enc)

we also need to build an observation dictionary.

In [10]:
observation = lca.build_observation(claims_enc)

### Model

Create some data

In [11]:
# claims = dict()
# claims['source_id'] = [0, 0, 1, 1]
# claims['object_id'] = [0, 1, 1, 0]
# claims['value'] = [0, 1, 0, 1]
# claims = pd.DataFrame(data=claims)

build inputs for simpleLCA model

In [12]:
# mask = lca.build_mask(claims)
# observation = lca.build_observation(claims)

# Inference 

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.infer
import pyro.optim
import pyro.distributions as dist

pyro.set_rng_seed(101)

In [14]:
data = lca.make_observation_mapper(observation, mask)
conditioned_lca = pyro.condition(lca.lca_model, data=data)

# guide
lca_guide = lca.lca_guide

pyro.clear_param_store()
svi = pyro.infer.SVI(model=conditioned_lca,
                     guide=lca_guide,
                     optim=pyro.optim.SGD({"lr": 0.001, "momentum":0.1}),
                     loss=pyro.infer.Trace_ELBO())

In [19]:
import torch

In [34]:
rv = dist.Categorical(logits=torch.log(torch.tensor([1.,1., -31.])))

In [35]:
rv.sample([100])

RuntimeError: invalid argument 2: invalid multinomial distribution (encountering probability entry < 0) at /Users/distiller/project/conda/conda-bld/pytorch_1565272679438/work/aten/src/TH/generic/THTensorRandom.cpp:356

In [28]:
from torch.distributions.utils import probs_to_logits, logits_to_probs, lazy_property


In [31]:
logits = probs_to_logits(torch.tensor([0.2, 0.8]))

In [36]:
logits

tensor([-1.6094, -0.2231])

In [38]:
probs = logits_to_probs(logits=torch.tensor([-12., -33.]))

In [40]:
dist.Categorical(probs=probs)

Categorical(probs: torch.Size([2]))

In [17]:
dist.Categorical(torch.random)

Init signature: dist.Categorical(probs=None, logits=None, validate_args=None)
Docstring:     
Wraps :class:`torch.distributions.categorical.Categorical` with
:class:`~pyro.distributions.torch_distribution.TorchDistributionMixin`.
File:           ~/miniconda3/envs/spectrum/lib/python3.6/site-packages/pyro/distributions/torch.py
Type:           ABCMeta
Subclasses:     


In [16]:
svi.step(observation,mask)

RuntimeError: invalid argument 2: invalid multinomial distribution (encountering probability entry < 0) at /Users/distiller/project/conda/conda-bld/pytorch_1565272679438/work/aten/src/TH/generic/THTensorRandom.cpp:356
Trace Shapes:  
 Param Sites:  
     beta_s_0  
     beta_s_1  
     beta_s_2  
     beta_s_3  
     beta_s_4  
     beta_s_5  
     beta_s_6  
     beta_s_7  
     beta_s_8  
     beta_s_9  
    beta_s_10  
    beta_s_11  
    beta_s_12  
    beta_s_13  
    beta_s_14  
    beta_s_15  
    beta_s_16  
    beta_s_17  
    beta_s_18  
    beta_s_19  
    beta_s_20  
    beta_s_21  
    beta_s_22  
    beta_s_23  
    beta_s_24  
    beta_s_25  
    beta_s_26  
    beta_s_27  
    beta_s_28  
    beta_s_29  
    beta_s_30  
    beta_s_31  
    beta_s_32  
    beta_s_33  
    beta_s_34  
    beta_s_35  
    beta_s_36  
    beta_s_37  
    beta_s_38  
    beta_s_39  
    beta_s_40  
    beta_s_41  
    beta_s_42  
    beta_s_43  
    beta_s_44  
    beta_s_45  
    beta_s_46 

In [18]:
losses = []
num_steps = 100
for t in range(num_steps):
    losses.append(svi.step(observation, mask))

RuntimeError: Expected p_in >= 0 && p_in <= 1 to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)
Trace Shapes:
 Param Sites:
     beta_s_0
Sample Sites:

> /Users/totucuong/miniconda3/envs/spectrum/lib/python3.6/site-packages/torch/distributions/bernoulli.py(89)sample()
     87         shape = self._extended_shape(sample_shape)
     88         with torch.no_grad():
---> 89             return torch.bernoulli(self.probs.expand(shape))
     90 
     91     def log_prob(self, value):



ipdb>  shape


torch.Size([])


ipdb>  u


...

ipdb>  shape


torch.Size([])


ipdb>  torch.bernoulli(self.probs.expand(shape))


*** RuntimeError: Expected p_in >= 0 && p_in <= 1 to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)


ipdb>  exit


In [ ]:
sns.tsplot(losses)

# Appendix

In [ ]:
def generate_one_simpleLCA_sample(observation, mask):
    tracer = pyro.poutine.trace(lca.lca_model)
    trace = tracer.get_trace(observation, mask)

    for name, node in trace.nodes.items():
        if node['type'] == 'sample':
            print(f'{node["name"]} - sampled value {node["value"]} ')
    return trace

# for i in range(3):
#     generate_one_simpleLCA_sample(observation, mask)
#     print('-'*10)